In [8]:
from sklearn.manifold import MDS
import sys
import ipywidgets

from mathchords.io import ExperimentHandler
from mathchords.functions.characteristics import process

import mathchords.functions.gen_chords as gen
import mathchords.functions.clustering as cg
import mathchords.functions.characteristics as ce
import numpy as np
import mathchords.constans
from mathchords.constans import SCALES

In [9]:
reference_points=gen.Simula_inversiones(mathchords.constans.DO_chords)
len(reference_points)

46

In [10]:
def separate_chords(chords):
    triads = []
    tetrads = []
    
    for chord in chords:
        if len(chord['intervals']) == 2:  # Las triadas tienen 2 intervalos (3 notas en total)
            triads.append(chord)
        elif len(chord['intervals']) == 3:  # Los acordes de 4 notas tienen 3 intervalos (4 notas en total)
            tetrads.append(chord)
    
    return triads, tetrads

#triads, tetrads=separate_chords(mathchords.constans.DO_chords)
triads, tetrads=separate_chords(reference_points)



In [11]:
BASE_PATH = "../data/"
handler = ExperimentHandler(BASE_PATH)
#datos=ce.create_experiment_dict(triads)
datos=ce.create_experiment_dict(tetrads)

chords=datos['chords']
results_structure=ce.process( datos, ce.transpose_to_zero)['results']
results_intervals_theory=ce.process(datos, ce.interval_vector)['results']
results_intervals_histo_12=ce.process(datos, ce.interval_histogram_with_dissmeasure)['results']
results_binary_notes=ce.process(datos, ce.binary_pitch_class_set)['results']
results_binary_intervals_12=ce.process(datos, ce.interval_histogram)['results']
results_polar_notes=ce.process(datos, ce.polar_pitch_classes)['results']
#results_polar_degree=ce.polar_degree(datos)
#print(results_intervals_theory.keys())
chord_ids = list(results_intervals_theory.keys())
def perform_mds(data, n_components=2):
    mds = MDS(n_components=n_components,  max_iter=30000,eps=1e-16)
    val=mds.fit_transform(data)
    print(mds.n_iter_,mds.stress_)
    return val,mds.dissimilarity_matrix_

In [12]:
vector_encoding = [
    #results_structure[chord_id]['feature_vector'] 
    #+
      results_intervals_theory[chord_id]['feature_vector']
    for chord_id in results_intervals_theory.keys()
]

print(vector_encoding)
# Realiza MDS en los datos
vector_encoding_mds,matrix =ce.perform_mds2(vector_encoding)
cg.plot_chords_with_selection(vector_encoding_mds,matrix, chord_ids, results_intervals_theory,True )



[[1, 0, 1, 2, 2, 0], [1, 0, 1, 2, 2, 0], [1, 0, 1, 2, 2, 0], [1, 0, 1, 2, 2, 0], [0, 1, 2, 1, 2, 0], [0, 1, 2, 1, 2, 0], [0, 1, 2, 1, 2, 0], [0, 1, 2, 1, 2, 0], [0, 1, 2, 1, 1, 1], [0, 1, 2, 1, 1, 1], [0, 1, 2, 1, 1, 1], [0, 1, 2, 1, 1, 1], [0, 1, 2, 1, 1, 1], [0, 1, 2, 1, 1, 1], [0, 1, 2, 1, 1, 1], [0, 1, 2, 1, 1, 1], [1, 0, 1, 3, 1, 0], [1, 0, 1, 3, 1, 0], [1, 0, 1, 3, 1, 0], [1, 0, 1, 3, 1, 0], [0, 2, 0, 3, 0, 1], [0, 2, 0, 3, 0, 1], [0, 2, 0, 3, 0, 1], [0, 2, 0, 3, 0, 1], [0, 0, 4, 0, 0, 2], [0, 0, 4, 0, 0, 2], [0, 0, 4, 0, 0, 2], [0, 0, 4, 0, 0, 2]]
Número de iteraciones: 424, Stress: 14.269508858381625
['CMaj7', 'CMaj7/E', 'CMaj7/G', 'CMaj7/B', 'Cm7', 'Cm7/D#', 'Cm7/G', 'Cm7/A#', 'C7', 'C7/E', 'C7/G', 'C7/A#', 'Cm7b5', 'Cm7b5/D#', 'Cm7b5/F#', 'Cm7b5/A#', 'CmM7', 'CmM7/D#', 'CmM7/G', 'CmM7/B', 'C7#5', 'C7#5/E', 'C7#5/G#', 'C7#5/A#', 'Cdim7', 'Cdim7/D#', 'Cdim7/F#', 'Cdim7/A']


Dropdown(description='Chord ID:', options=(('Chord ID: chord_0, CMaj7', 'chord_0'), ('Chord ID: chord_1, CMaj7…

FigureWidget({
    'data': [{'hoverinfo': 'text',
              'hovertext': [Root: C<br>Bass: C<br>Octave: 4<br>Degree:
                            1<br>Intervals: [4, 3, 4], Root: C<br>Bass:
                            E<br>Octave: 4<br>Degree: 1<br>Intervals: [4, 3, 4],
                            Root: C<br>Bass: G<br>Octave: 4<br>Degree:
                            1<br>Intervals: [4, 3, 4], Root: C<br>Bass:
                            B<br>Octave: 4<br>Degree: 1<br>Intervals: [4, 3, 4],
                            Root: C<br>Bass: C<br>Octave: 4<br>Degree:
                            1<br>Intervals: [3, 4, 3], Root: C<br>Bass:
                            D#<br>Octave: 4<br>Degree: 1<br>Intervals: [3, 4, 3],
                            Root: C<br>Bass: G<br>Octave: 4<br>Degree:
                            1<br>Intervals: [3, 4, 3], Root: C<br>Bass:
                            A#<br>Octave: 4<br>Degree: 1<br>Intervals: [3, 4, 3],
                            Root: C<br>Bass: C<br>Oc

In [14]:
vector_encoding5 = [

     [10*x for x in results_intervals_histo_12[chord_id]['feature_vector']]
     #results_polar_degree[chord_id]['feature_vector'] +
     #results_intervals_histo_12[chord_id]['feature_vector']
     #+
     #results_binary_intervals_12[chord_id]['feature_vector']
     +
     results_binary_notes[chord_id]['feature_vector']
    #results_polar_notes[chord_id]['feature_vector']
    for chord_id in results_intervals_theory.keys()
]
print(vector_encoding5)
# Realiza MDS en los datos
vector_encoding_mds5,matrix = ce.perform_mds2(vector_encoding5,'euclidean')
#vector_encoding_mds5_our, matrix2=ce.perform_mds2(vector_encoding5,'euclidean')
cg.plot_chords_with_selection(vector_encoding_mds5,matrix, chord_ids, results_binary_intervals_12,True)

[[0, 0, 4.741198056768631, 11.052071811356836, 0, 0, 1.494321831470006, 0, 0, 0, 0.016235255678833785, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1], [8.94974601785532, 0, 4.741198056768631, 1.822128650169923, 0.7798994489402729, 0, 0.22589395150038497, 0.0878685307938039, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1], [8.94974601785532, 0, 0, 5.597674905047843, 2.364643012106085, 0, 0, 0, 0.012172647761089065, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1], [8.94974601785532, 0, 1.737764418752922, 0.9767088023539988, 0.40242205711276957, 0, 0.03111923187083253, 0.011482466389364888, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1], [0, 0, 19.675143510714697, 2.8035027673272617, 0, 0, 1.6052871470218935, 0, 0, 0.04210130559153637, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0], [0, 6.125367131066717, 3.8338330344687943, 2.8035027673272617, 0.7798994489402729, 0, 0.2813766092763288, 0, 0.043528176471628974, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0], [0, 6.125367131066717, 12.73254658775748, 0, 2.5152639421955088, 

Dropdown(description='Chord ID:', options=(('Chord ID: chord_0, CMaj7', 'chord_0'), ('Chord ID: chord_1, CMaj7…

FigureWidget({
    'data': [{'hoverinfo': 'text',
              'hovertext': [Root: C<br>Bass: C<br>Octave: 4<br>Degree:
                            1<br>Intervals: [4, 3, 4], Root: C<br>Bass:
                            E<br>Octave: 4<br>Degree: 1<br>Intervals: [4, 3, 4],
                            Root: C<br>Bass: G<br>Octave: 4<br>Degree:
                            1<br>Intervals: [4, 3, 4], Root: C<br>Bass:
                            B<br>Octave: 4<br>Degree: 1<br>Intervals: [4, 3, 4],
                            Root: C<br>Bass: C<br>Octave: 4<br>Degree:
                            1<br>Intervals: [3, 4, 3], Root: C<br>Bass:
                            D#<br>Octave: 4<br>Degree: 1<br>Intervals: [3, 4, 3],
                            Root: C<br>Bass: G<br>Octave: 4<br>Degree:
                            1<br>Intervals: [3, 4, 3], Root: C<br>Bass:
                            A#<br>Octave: 4<br>Degree: 1<br>Intervals: [3, 4, 3],
                            Root: C<br>Bass: C<br>Oc